# Generazione dati progetto di laboratorio

## Importo le librerie

In [1]:
import pandas as pd
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine, text, MetaData
import random 
import datetime
from pprint import pprint
import random
import numpy as np
fake = Faker('it-IT')

Definisco le costanti e i file da dove importare i dati:

In [2]:
NUM_FORNITORI = 1000
NUM_DIPENDENTI = 10000
NUM_MATRIMONI = 100
NUM_PROGETTI = 5000
NUM_COMPETENZE_DIP = 10
NUM_DIP_PROGETTO = 5

FILE_DIPARTIMENTO = "MOCK_DEPARTMENT.txt"
FILE_QUALIFICHE = "MOCK_QUALIFY.txt"
FILE_SKILLS = "MOCK_SKILLS.txt"
FILE_LAUREE = "MOCK_DEGREES.txt"
FILE_DOTTORATI = "MOCK_PHDS.txt"

Relazione Fornitore

In [3]:


#FORNITORE
fornitore = defaultdict(list)
for _ in range(NUM_FORNITORI):
    fornitore["nome"].append(fake.company())
    fornitore["indirizzo"].append(fake.address())

df_fornitore = pd.DataFrame(fornitore)
df_fornitore.drop_duplicates(inplace=True, subset="nome")

print("Created Fornitore")


Created Fornitore


Relazione Dipartimento

In [4]:
#DIPARTIMENTO

dipartimento = defaultdict(list)

f = open(FILE_DIPARTIMENTO)
lines = f.read().splitlines()
f.close()

dipartimento["recapito"] = (fake.random_sample(elements = range(100000,999999), length = len(lines)))

for i in range(len(lines)):
    dipartimento["recapito"][i] = int("0432" + str(dipartimento["recapito"][i]))

for line in lines:
    dipartimento["nome"].append(line)
    dipartimento["email"].append(line.replace(" ","").lower() + "@company.com")
    dipartimento["ultimo_acquisto"].append(random.choice(fornitore["nome"]))
    dipartimento["data_ultimo_acquisto"].append(fake.date_between(start_date = datetime.date(2021,1,1)))

#Numero afferenti viene aggiunto dopo

print("Created Dipartimento")

df_dipartimento = pd.DataFrame(dipartimento)

#df_dipartimento.to_csv("dipartimento.csv")


Created Dipartimento


Relaazione fornisce

In [5]:
#FORNISCE

fornisce = defaultdict(list)

#Ogni dipartimento ha la probabilità del 90% di essere rifornito dal suo ultimo acquisto
for i in range(len(dipartimento["nome"])):
    p = random.randint(0,9)
    if p < 9:
        fornisce["fornitore"].append(dipartimento["ultimo_acquisto"][i])
        fornisce["dipartimento"].append(dipartimento["nome"][i])


# Selezioniamo 10 fornitori fortunati che riforniranno la maggior parte dei dipartimenti
fornitori_fortunati = fake.random_sample(elements=range(0, len(fornitore["nome"])), length=10)

#2/12 (arbitrario) aggiungiamo un elemento in fornisce con un fornitore non fortunato
fornitori_fortunati.append(-1)
fornitori_fortunati.append(-1)


for _ in range(400):
    #Scegliamo chi sarà il fornitore seguendo una distribuzione normale
    fornitore_scelto = int(abs(np.random.normal(0, 10))%12)
    if fornitori_fortunati[fornitore_scelto] == -1:  
        fornisce["fornitore"].append(fake.random_element(elements=fornitore["nome"]))
    else:
        fornisce["fornitore"].append(fornitore["nome"][fornitori_fortunati[fornitore_scelto]])
    
    fornisce["dipartimento"].append(fake.random_element(dipartimento["nome"]))

print("Created Fornisce")

df_fornisce = pd.DataFrame(fornisce)
df_fornisce.drop_duplicates(inplace=True)



Created Fornisce


In [6]:
#DIPENDENTE

#Scegliamo chi saranno i dipartimenti fortunati ad avere la maggior parte dei dipendenti
dipartimenti_fortunati = fake.random_sample(elements=range(0, len(dipartimento["nome"])), length=10)

#2/12 (arbitrario) aggiungiamo un dipendente che afferisce ad un dipartimento non fortunato

dipartimenti_fortunati.append(-1)
dipartimenti_fortunati.append(-1)

dipendente = defaultdict(list)

dipendente["matricola"] = fake.random_sample(elements = range(1, 1000000), length = NUM_DIPENDENTI)

f = open(FILE_QUALIFICHE)
qualifies = f.read().splitlines()
f.close()

f = open(FILE_LAUREE)
degrees = f.read().splitlines()
f.close()

f = open(FILE_DOTTORATI)
phds = f.read().splitlines()
f.close()


matricole_laurea = []
matricole_dottorato = []

#Vettore con l'età dei dipendenti, NON viene aggiunta alla base di dati, serve ad usi successivi dello script
eta_dipendenti = []

#Funzione che calcola l'età data una data di nascita
def calculateAge(birthDate):
    today = datetime.date.today()
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))
 
    return age

for i in range(NUM_DIPENDENTI):
    dipendente["nome"].append(fake.first_name())
    dipendente["cognome"].append(fake.last_name())
    dipendente["qualifica"].append(fake.random_element(elements=qualifies))

    dipartimento_scelto = int(abs(np.random.normal(0, 10))%12)
    if dipartimenti_fortunati[dipartimento_scelto] == -1:  
        dipendente["dipartimento"].append(fake.random_element(elements=dipartimento["nome"]))
    else:
        dipendente["dipartimento"].append(dipartimento["nome"][dipartimenti_fortunati[dipartimento_scelto]])
    
    dipendente["data_assunzione"].append(fake.date_time_between(start_date = datetime.date(2015, 1, 1)))

    nascita = fake.date_time_between(end_date = datetime.date(2003, 1, 1))
    dipendente["data_di_nascita"].append(nascita)
    eta_dipendenti.append(calculateAge(nascita))

    p_laureato = random.randint(0,9)
    if p_laureato < 5:
        dipendente["classe_laurea"].append(fake.random_element(elements = degrees))
        dipendente["data_laurea"].append(fake.date_time_between(start_date = nascita + datetime.timedelta(days = 20*365)))
        matricole_laurea.append(dipendente["matricola"][i])

        p_dottorato = random.randint(0,9)
        if p_dottorato < 2:
            dipendente["classe_dottorato"].append(fake.random_element(elements = phds))
            dipendente["data_dottorato"].append(fake.date_time_between(start_date = nascita + datetime.timedelta(days = 28*365)))
            matricole_dottorato.append(dipendente["matricola"][i])

        else:
            dipendente["classe_dottorato"].append(None)
            dipendente["data_dottorato"].append(None)
    else:
        dipendente["classe_laurea"].append(None)
        dipendente["data_laurea"].append(None)
        dipendente["classe_dottorato"].append(None)
        dipendente["data_dottorato"].append(None)


df_dipendente = pd.DataFrame(dipendente)

print("Created Dipendente")

df_dipartimento.insert(2, "numero_afferenti", 0)

#df_dipartimento.set_index("nome", inplace=True)

for i in range(len(df_dipendente.index)):
    dip = df_dipendente.iloc[[i]]["dipartimento"].to_string(index = False)
    df_dipartimento.loc[df_dipartimento["nome"] == dip, "numero_afferenti"] +=  1

 Sustainability
 Innovation
 Customer Service
 User Experience
 Customer Success
 Artificial Intelligence
 Event Management
 Research and Innovation
 Corporate Training
 Corporate Development
Created Dipendente


In [7]:
#df_dipendente.to_csv("dipendente.csv")
#df_dipartimento.to_csv("dipartimento.csv")

In [8]:
#Matrimonio

matrimonio = defaultdict(list)

coniugi = fake.random_sample(elements = dipendente["matricola"], length = NUM_MATRIMONI*2)

for i in range(NUM_MATRIMONI):
    matrimonio["coniuge1"].append(coniugi[2*i])
    matrimonio["coniuge2"].append(coniugi[2*i+1])

print("Created Matrimonio")

df_matrimonio = pd.DataFrame(matrimonio)

Created Matrimonio


In [9]:
#PROGETTO

progetto = defaultdict(list)

progetto["codice_aziendale"] = fake.random_sample(elements = range(1000000), length = NUM_PROGETTI)
progetto["budget"] = fake.random_elements(elements = range(1000, 50000), unique = False, length = NUM_PROGETTI)
progetto["durata_in_mesi"] = fake.random_elements(elements = range(6, 48), unique = False, length=NUM_PROGETTI)

print("Created Progetto")

df_progetto = pd.DataFrame(progetto)

Created Progetto


In [10]:
#COMPETENZA

competenza = defaultdict(list)


f = open(FILE_SKILLS)
skills = f.read().splitlines()
f.close()

competenza["nome"] = skills

print("Created Competenza")

df_competenza = pd.DataFrame(competenza)
df_competenza.drop_duplicates(inplace=True)

Created Competenza


In [11]:
#POSSIEDE

possiede = defaultdict(list)

def add_possiede(i):
    possiede["competenza"].append(fake.random_element(competenza["nome"])) 
    p = random.randint(0,9)
    if p < 3:
        possiede["matricola"].append(fake.random_element(matricole_dottorato))
    elif p < 6:
        possiede["matricola"].append(fake.random_element(matricole_laurea))
    else:
        possiede["matricola"].append(fake.random_element(dipendente["matricola"]))
    if(i % 10000 == 0):
        print(str(i) + " elementi in possiede creati")


for i in range(NUM_COMPETENZE_DIP*NUM_DIPENDENTI):
    add_possiede(i)

df_possiede = pd.DataFrame(possiede)
df_possiede.drop_duplicates(inplace = True)


print("Created Possiede")

0 elementi in possiede creati
10000 elementi in possiede creati
20000 elementi in possiede creati
30000 elementi in possiede creati
40000 elementi in possiede creati
50000 elementi in possiede creati
60000 elementi in possiede creati
70000 elementi in possiede creati
80000 elementi in possiede creati
90000 elementi in possiede creati
Created Possiede


In [12]:
#COINVOLGE

coinvolge = defaultdict(list)

for i in range(len(progetto["codice_aziendale"])):
    p = progetto["codice_aziendale"][i]

    #Dipendenza lineare tra budget e num dipendenti del progetto

    budget = progetto["budget"][i]
    coeff = 2 * NUM_DIP_PROGETTO / 50000
    noise = int(np.random.normal(0,0.5)) # La deviazione standard del noise è arbitraria...
    #TO DO: aggiungere dipendenza da età
    num_dip = int(coeff * budget) + noise
    num_dip = num_dip * (num_dip > 0)
    matr = fake.random_sample(dipendente["matricola"], length = num_dip)
    for i in range(num_dip):
        competenze  = df_possiede.loc[df_possiede['matricola'] == matr[i]]["competenza"].tolist()
        if(competenze != []):
            coinvolge["matricola"].append(matr[i])
            coinvolge["competenza"].append(fake.random_element(competenze))
            coinvolge["progetto"].append(p)
        #coinvolge["competenza"].append(fake.random_element(posssiede))
        
print("Created Coinvolge")  

df_coinvolge = pd.DataFrame(coinvolge)

Created Coinvolge


In [13]:
engine = create_engine("postgresql+psycopg2://postgres:postgres@/progetto_basi")

with engine.connect() as con:
    f = open("../SQL/data_definition.sql")
    data_definition = text(f.read())
    f.close()

    f = open("../SQL/trigger.sql")
    trigger = text(f.read())
    f.close()

    f = open("../SQL/indexes.sql")
    indici = text(f.read())
    f.close()
    
    con.execute(data_definition)
    con.execute(trigger)
    con.execute(indici)
    con.commit()

In [14]:

df_fornitore.to_sql('fornitore', con=engine, index=False, if_exists='append')
df_dipartimento.to_sql('dipartimento', con=engine, index=False, if_exists='append')
df_fornisce.to_sql('fornisce', con=engine, index=False, if_exists='append')
df_dipendente.to_sql('dipendente', con=engine, index=False, if_exists='append')
df_matrimonio.to_sql('matrimonio', con=engine, index=False, if_exists='append')
df_progetto.to_sql('progetto', con=engine, index=False, if_exists='append')
df_competenza.to_sql('competenza', con=engine, index=False, if_exists='append')
df_possiede.to_sql('possiede', con=engine, index=False, if_exists='append')
df_coinvolge.to_sql('coinvolge', con=engine, index=False, if_exists='append')

476